In [108]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [109]:
import pandas as pd
import pickle as pkl

In [110]:
%run utils.ipynb

In [124]:
df = pd.read_csv('data/ETHBTC_1m_1519496760000_1549568220000.csv')
df.shape
df.head(10)

(500000, 12)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE  VOLUME  \
0  1519496760000  0.085541  0.085570  0.085407  0.085473  27.714   
1  1519496820000  0.085473  0.085570  0.085448  0.085449  20.128   
2  1519496880000  0.085448  0.085569  0.085400  0.085538  45.510   
3  1519496940000  0.085538  0.085546  0.085402  0.085540  22.231   
4  1519497000000  0.085539  0.085552  0.085464  0.085464  15.243   
5  1519497060000  0.085466  0.085555  0.085464  0.085545  31.895   
6  1519497120000  0.085545  0.085545  0.085464  0.085539  13.325   
7  1519497180000  0.085539  0.085555  0.085414  0.085517  28.202   
8  1519497240000  0.085460  0.085545  0.085419  0.085534  15.302   
9  1519497300000  0.085440  0.085545  0.085440  0.085533  13.730   

      CLOSE_TIME  QUOTE_VOL  NUM_TRADES  TAKE_BASE_VOL  TAKE_QUOTE_VOL  EXTRA  
0  1519496819999   2.370345         192         15.308        1.309733      0  
1  1519496879999   1.720609         122          6.579        0.562654      0  
2  1519496939999   3.890176         209         19.426        1.660960      0  
3  1519496999999   1.900513         105         14.320        1.224483      0  
4  1519497059999   1.303412         117          8.373        0.716219      0  
5  1519497119999   2.727413         131         19.237        1.645547      0  
6  1519497179999   1.139346          78          7.311        0.625352      0  
7  1519497239999   2.411093         195         14.409        1.232346      0  
8  1519497299999   1.307959         134          7.216        0.617035      0  
9  1519497359999   1.173580         105          6.186        0.529001      0

In [125]:
data = df.loc[:, ['OPEN_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.shape
data[:5]

(500000, 5)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE
0  1519496760000  0.085541  0.085570  0.085407  0.085473
1  1519496820000  0.085473  0.085570  0.085448  0.085449
2  1519496880000  0.085448  0.085569  0.085400  0.085538
3  1519496940000  0.085538  0.085546  0.085402  0.085540
4  1519497000000  0.085539  0.085552  0.085464  0.085464

In [126]:
data_validation(data)

In [127]:
# Get Target
%run targets.ipynb
target = get_target_1()
target.shape
target.head()

(499999, 2)

OPEN_TIME  TARGET
0  1519496760000     1.0
1  1519496820000     0.0
2  1519496880000     0.0
3  1519496940000     1.0
4  1519497000000     0.0

In [128]:
data = pd.merge(data, target, on='OPEN_TIME')
data.shape

(499999, 6)

In [129]:
data_validation(data)

In [130]:
frac_data_use = 0.2

tr_frac = 0.6
cv_frac = 0.2
tt_frac = 0.2

TR_data = data[:int(len(data)*tr_frac)]
CV_data = data[int(len(data)*tr_frac):int(len(data)*(tr_frac+cv_frac))]
TT_data = data[int(len(data)*(tr_frac+cv_frac)):]

TR_data.shape
CV_data.shape
TT_data.shape

(299999, 6)

(100000, 6)

(100000, 6)

In [131]:
win = 500
TR_X, TR_y = get_multiple_ts_dataset(TR_data.values, tr_win=win, tt_win=0, point_target=True)
CV_X, CV_y = get_multiple_ts_dataset(CV_data.values, tr_win=win, tt_win=0, point_target=True)
TT_X, TT_y = get_multiple_ts_dataset(TT_data.values, tr_win=win, tt_win=0, point_target=True)

In [132]:
TR_X.shape
TR_y.shape

(299500, 2001)

(299500,)

In [ ]:
from sklearn.linear_model import BayesianRidge

model = BayesianRidge(n_iter=300, tol=0.001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06, compute_score=True, fit_intercept=True, normalize=True, copy_X=True, verbose=True)
model = model.fit(TR_X, TR_y)

In [49]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB().fit(TR_X, TR_y.astype(int))

In [105]:
TR_y_pred = model.predict(TR_X)

In [106]:
TR_y_pred[:5]

array([0.50761034, 0.50775131, 0.50779143, 0.50787017, 0.50791823])

In [107]:
%run targets.ipynb
eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

{'acc': 0.5120612363148425,
 'auc': 0.510105316224727,
 'f1': 0.3783993783993784,
 'log_loss': 16.85292068115887}

In [92]:
%run targets.ipynb
eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

{'acc': 0.5089331912950844,
 'auc': 0.5003447968069276,
 'f1': 0.011073880353056554,
 'log_loss': 16.960848696468673}

In [3]:
import pandas as pd
pd.Series({'a':11, 'b':2})

a    11
b     2
dtype: int64